# Иллюстрируем способ использования предсказания по ряду поправок


### Импортируем модули

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
import time
from processing import *

### Загружаем данные заказов и тарифов

In [3]:
snap_shot_file = 'stat_2024_09_12.csv'
pr_df = prepare_prices("тарифы июнь 2024_1.csv")
transfers = pd.read_csv(snap_shot_file,delimiter=';')
transfers['ctime'].min(), transfers['ctime'].max()

('2024-06-23 18:57:16.949313', '2024-09-12 00:15:58.466253')

#### Приводим типы и берём данные за июль-сентябрь и расстояния не ближе 50 км

In [4]:
adjust_types(transfers)
transfers = transfers.sort_values(by=['ctime','delivery_order_id'])
transfers = transfers[(transfers['ctime']>='2024-07-01 00:00:00') & (transfers['distance']>=50)].copy()
wdf, dest = make_working_df(transfers,target_key='price_gross_per_ton',add_month=True,adjust_dist=False, diesel_file=None,diesel_num=1)
wdf.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 726 entries, (Timestamp('2024-07-01 00:00:00'), 47871) to (Timestamp('2024-09-12 00:00:00'), 52016)
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   month        726 non-null    int32  
 1   dist         726 non-null    int64  
 2   target       726 non-null    int64  
 3   targ_price   726 non-null    float64
 4   region_id    726 non-null    int64  
 5   crops_id     726 non-null    int64  
 6   day_of_week  726 non-null    int32  
 7   hours        726 non-null    int64  
 8   dest_id      726 non-null    int64  
dtypes: float64(1), int32(2), int64(6)
memory usage: 72.1 KB


In [5]:
dest

array(['«ЗТК Тамань» ООО', 'АО "ЕЙСКИЙ ПОРТОВЫЙ ЭЛЕВАТОР"',
       'АО "КХП "ТИХОРЕЦКИЙ"', 'АО "ПЛЕМЕННОЙ ЗАВОД "ГАШУНСКИЙ"',
       'АО "РОСТОВСКИЙ ПОРТ" (Четвертый грузовой район)',
       'АО «Ейский морской порт»', 'АО «КСК»', 'АО РАССВЕТ', 'Азов',
       'Архангельский Морской Торговый Порт',
       'Багаевское ХПП Агропром-Импэкс, филиал (Агропромимпэкс, Элеватор)',
       'Гигант', 'ЗЕРНОВОЙ ТЕРМИНАЛ "СТЕПЬ"', 'Зерновой терминал «СТЕПЬ»',
       'Новатор', 'ООО  ПКФ «Порт Корвет»', 'ООО "ВИРТУМ" (Азов)',
       'ООО "ВНЕШТОРГПОРТ"', 'ООО "КАНЕВСКОЙ ЭЛЕВАТОР"',
       'ООО "МАСЛОЗАВОД КРОНОС"',
       'ООО "МОРСКОЙ ГРУЗОВОЙ ТЕРМИНАЛ ИМПЕРИАЛ"',
       'ООО "МЭЗ" (филиал Гигант)', 'ООО "РОСТОВСКИЙ ЗЕРНОВОЙ ТЕРМИНАЛ"',
       'ООО "ЮГ РУСИ"', 'ООО "ЮЖНЫЙ МОРСКОЙ ПОРТ"', 'ООО «НЗТ»',
       'ООО Благодарное', 'ООО ИПАТОВСКИЙ КОМБИНАТ ХЛЕБОПРОДУКТОВ',
       'ООО ЛАБИНСКИЙ ЭЛЕВАТОР', 'ООО МЭЗ Ресурс ОП Усть-Лабинский МЭЗ',
       'ООО ПТ ПРИБОЙ', 'ООО ЧЕРНЫШКОВСКИЙ ЭЛЕВАТОР', 'ПАО «Н

Очищаем набор данных от выбросов и от перевозок из регионов, для которых отсутсвуют тарифы

In [6]:
wdf = refine_df(wdf,pr_df)

#### Считаем количество данных по культурам и регионам

In [7]:
counts = wdf.groupby(['region_id','crops_id']).size().reset_index().rename(columns={0: 'count'})
counts

,region_id,crops_id,count
0,8,1,2
1,8,2,1
2,23,1,13
3,23,2,371
4,23,3,13
5,23,6,71
6,23,8,1
7,23,16,1
8,23,24,2
9,26,1,15


In [8]:
# посмотрим, для каких сочетаний регион-культура имеет смысл применять времнные ряды из наличия хотя бы пары перевозок на 3 дня
# грубо 74 дня, делим на 3 и умножаем на 2
# соотношение "на грани", лучше - больше 
# в остальных случаях будем использовать только прецеденты
counts[counts['count']>50]

,region_id,crops_id,count
3,23,2,371
5,23,6,71
10,26,2,74
28,61,2,78


In [9]:
wdf.tail(20)

month  dist  target   targ_price  region_id  crops_id  \
date       id                                                             
2024-09-11 51859      9   345  130000   376.811594         23         2   
           51887      9   423  150000   354.609929         23         2   
           51907      9   390  164000   420.512821         23         2   
           51911      9   595  128000   215.126050         29         1   
           51924      9    60   94000  1566.666667         23         3   
2024-09-12 51799      9   260  146000   561.538462         23         2   
           51818      9   235  120000   510.638298         23         2   
           51864      9   362  140000   386.740331         23         2   
           51865      9   252  125000   496.031746         23         2   
           51869      9   323  125000   386.996904         23         2   
           51904      9   371  160000   431.266846         23         2   
           51910      9   226  120000   530.973451         23         2   
           51920      9   403  175000   434.243176         26         2   
           51948      9   222  110000   495.495495         23         2   
           51954      9   267  130000   486.891386         23         2   
           51979      9   341  150000   439.882698         23         2   
           51982      9   283  120000   424.028269         23         2   
           51998      9   601  220000   366.056572         26         1   
           52003      9   365  150000   410.958904         23         3   
           52016      9   505  190000   376.237624         61         2   

                  day_of_week  hours  dest_id  
date       id                                  
2024-09-11 51859            2     24        0  
           51887            2     20        0  
           51907            2      3        6  
           51911            2      6       33  
           51924            2      0       32  
2024-09-12 51799            3     68       32  
           51818            3      5       25  
           51864            3     43       25  
           51865            3     38        6  
           51869            3     45        0  
           51904            3      0       25  
           51910            3     26       25  
           51920            3     23        6  
           51948            3     22       25  
           51954            3     20       32  
           51979            3     15       25  
           51982            3      0        0  
           51998            3      0       32  
           52003            3      3       25  
           52016            3      2        0

### Задаём наборы параметров фильтрации и моделей

In [10]:
fp = filter_par_df(np.arange(1,4),np.append(np.arange(2,11),0),np.append(np.arange(2,6),0))
fp

,use_filter,window,filter_window,filter_order
0,True,1,3,2
1,True,1,4,2
2,True,1,4,3
3,True,1,5,2
4,True,1,5,3
...,...,...,...,...
76,True,3,10,4
77,True,3,10,5
78,False,1,0,0
79,False,2,0,0


In [11]:
mp = model_par_df(['fourier','poly'],np.arange(2,20),np.arange(0,10),np.append(np.arange(4,9),0),np.arange(0,7))
mp

,variant,train_window,n_harm,period,poly_order
0,fourier,2,1,0,0
1,fourier,3,1,0,0
2,fourier,3,2,0,0
3,fourier,4,1,0,0
4,fourier,4,2,0,0
...,...,...,...,...,...
145,poly,8,0,0,5
146,poly,8,0,0,6
147,poly,9,0,0,5
148,poly,9,0,0,6


### Задаём функцию предсказания с использованием временных рядов по принципу скользящего окна

In [12]:
#предсказание идёт на одну дату с заданным смещением (горизонтом прогнозирования) относительно конца обучающей выборки
#затем окно выборки и дата предсказания смещаются

#pred_start - дата начала предсказаний,
#t_df - набор данных с тарифами,
#region_id,crops_id - выбранные регион и культура,
#horizon - горизонт прогнозирования,
#filt_pars - параметры фильтрации,
#model_pars - параметры модели прогноза,
#form_type - тип формулы цены,
#per_tonn_km - признак расчёта удельной цены
#prefix - префикс названия столбца результата
#min_samples - минимально требуемое число перевозок для подбора коэффициентов
#min_dists - минимально требуемое число перевозок на разные дистанции для подбора коэффициентов
def slide_predict(df,*,form_type,min_samples,min_dists,pred_start,t_df,region_id,crops_id,horizon,\
                  filt_pars, model_pars,per_tonn_km = 'True',prefix = ''):
    #создаём экземпляр модели
    predictor = SeriesPredictor(tarif_df = pr_df,fpars = filt_pars, mpars = mod_pars, region_id = region_id, form_type=form_type)
    #делаем выборку по региону и культуре
    rdf = df_region_crop(df,region_id = region_id, crops_id = crops_id)
    #распаковываем параметры
    _,sample_window,_,_ = predictor.fpars
    _, train_window, _, _, _ = predictor.mpars
    #считаем даты и дельты
    #смещение конца обучающей выборки относительно начала
    train_delta = pd.Timedelta(days = sample_window+train_window-2)
    #смещение даты предсказания относительно конца обучающей выборки
    test_delta = pd.Timedelta(days = horizon)
    #дата начала самой первой обучающей выборки
    start_date =  pd.to_datetime(pred_start) - train_delta - test_delta
    #дата последнего предсказания
    pred_end = rdf.index.get_level_values(0).max()
    #дата начала последней обучающей выборки
    end_date = pred_end - train_delta - test_delta
    #диапазон дат начала обучающих выборок
    date_range = pd.date_range(start_date,end_date)
    #диапазон дат предсказания
    pred_range = pd.date_range(pred_start,pred_end)
    #наименование столбца результата
    res_key = pred_key(prefix,horizon)
    rdf[res_key] = None
    #цикл сдвига окна
    for date in date_range:
        #формируем тестовый набор на горизонт прогнозирования
        test_date = date + train_delta + test_delta
        test_df = rdf[rdf.index.get_level_values(0)==test_date].copy()
        #print(test_date,len(test_df))
        #если есть, что предсказывать
        if len(test_df)>0:            
            #формируем обучающий набор данных
            train_end = date + train_delta;
            #print(date,train_end)
            train_df = rdf[(rdf.index.get_level_values(0)>=date)&(rdf.index.get_level_values(0)<=train_end)].copy()
            train_df.reindex(pd.date_range(date,train_end),level = 0)
            #print(train_df)
            try:
                predictor.fit(train_df,date = train_end)
                #рассчитываем результат на дату прогноза
                res = predictor.predict(horizon,horizon, df = test_df)
            except:
                print('не хватает данных для прогноза на дату',test_date)
            else:

                #при необходимости, приводим к коп/(т*км)
                if per_tonn_km:
                    res['target_pred'] = res['target_pred']/res['dist']
                #заносим в соответствующий столбец
                rdf.loc[res.index,[res_key]] = res['target_pred']
    if per_tonn_km:
        rdf['err'+res_key] = rdf[res_key]-rdf['targ_price']
    else:
        rdf['err_'+res_key] = rdf[res_key]-rdf['target']
    return rdf

### Применяем вариант 3 модели на основе ряда коээфициентов

<hr>
<i> Предсказание по формуле $p(d) = k_1\times d \times t(d)$</i>
<hr>

#### Загружаем параметры фильтрации и модели предсказания

In [13]:
filt_pars, mod_pars = get_pars(fp,mp,52,127)
filt_pars, mod_pars

(use_filter       True
 window              3
 filter_window       3
 filter_order        2
 Name: 52, dtype: object,
 variant         poly
 train_window       3
 n_harm             0
 period             0
 poly_order         1
 Name: 127, dtype: object)

<hr>
    <i>
        Параметры фильтрации:
        <ul>
            <li> размер окна для оценки коэффициентов: 3 дня </li>
            <li> использовать полиномиальную (sav_gol) фильтрацию: да </li>
            <li> размер окна фильтра: 3 дня </li>
            <li> порядок фильтра: 2 </li>
        </ul>
    </i>
<hr>
    <i>
        Параметры модели предсказания:
        <ul>
            <li> тип: полином </li>
            <li> размер окна для анализа: 3 дня </li>
            <li> порядок полинома: 1 (линейный) </li>
        </ul>
    </i>

<hr>

In [16]:
var3_23_2 = slide_predict(wdf,form_type = 'k1',min_samples =2,min_dists=2,pred_start = '2024-08-01',t_df = pr_df,region_id = 23,crops_id = 2,horizon = 3,\
                  filt_pars = filt_pars, model_pars = mod_pars ,per_tonn_km = 'True',prefix = 'var3')
var3_23_2.tail(50)

month  dist  target   targ_price  region_id  crops_id  \
date       id                                                             
2024-09-05 51558      9   392  160000   408.163265         23         2   
           51576      9   414  163900   395.893720         23         2   
           51585      9   440  170000   386.363636         23         2   
2024-09-06 51564      9   226  120000   530.973451         23         2   
           51578      9   240  110000   458.333333         23         2   
           51607      9   163   93300   572.392638         23         2   
           51633      9   284  120000   422.535211         23         2   
           51667      9   191   95000   497.382199         23         2   
2024-09-07 51640      9    94   75000   797.872340         23         2   
           51694      9   353  165000   467.422096         23         2   
           51739      9   284  127000   447.183099         23         2   
           51740      9   215  107000   497.674419         23         2   
           51742      9   392  159000   405.612245         23         2   
           51744      9   320  130000   406.250000         23         2   
           51745      9   240  115000   479.166667         23         2   
2024-09-08 51668      9   382  140000   366.492147         23         2   
           51738      9   222  110000   495.495495         23         2   
           51752      9   382  163000   426.701571         23         2   
           51757      9   362  150000   414.364641         23         2   
2024-09-09 51602      9   294  124000   421.768707         23         2   
           51791      9   337  133000   394.658754         23         2   
           51798      9   359  163000   454.038997         23         2   
           51806      9   365  150000   410.958904         23         2   
2024-09-10 51588      9   246  130000   528.455285         23         2   
           51755      9   370  150000   405.405405         23         2   
           51802      9   317  127000   400.630915         23         2   
           51815      9   323  135000   417.956656         23         2   
           51822      9   148   85000   574.324324         23         2   
           51835      9   249  132000   530.120482         23         2   
           51897      9   291  110000   378.006873         23         2   
2024-09-11 51809      9    69   91000  1318.840580         23         2   
           51819      9   279  120000   430.107527         23         2   
           51824      9   253  120000   474.308300         23         2   
           51852      9   246  105000   426.829268         23         2   
           51854      9   337  127000   376.854599         23         2   
           51858      9   116   85000   732.758621         23         2   
           51859      9   345  130000   376.811594         23         2   
           51887      9   423  150000   354.609929         23         2   
           51907      9   390  164000   420.512821         23         2   
2024-09-12 51799      9   260  146000   561.538462         23         2   
           51818      9   235  120000   510.638298         23         2   
           51864      9   362  140000   386.740331         23         2   
           51865      9   252  125000   496.031746         23         2   
           51869      9   323  125000   386.996904         23         2   
           51904      9   371  160000   431.266846         23         2   
           51910      9   226  120000   530.973451         23         2   
           51948      9   222  110000   495.495495         23         2   
           51954      9   267  130000   486.891386         23         2   
           51979      9   341  150000   439.882698         23         2   
           51982      9   283  120000   424.028269         23         2   

                  day_of_week  hours  dest_id   var3_pred3 errvar3_pred3  
date       id                             

In [20]:
var3_23_2.to_csv('var3_23_2.csv',sep = ';')

In [21]:
var3_23_6 = slide_predict(wdf,form_type = 'k1',min_samples =2,min_dists=2,pred_start = '2024-08-01',t_df = pr_df,region_id = 23,crops_id = 6,horizon = 3,\
                  filt_pars = filt_pars, model_pars = mod_pars ,per_tonn_km = 'True',prefix = 'var3')
var3_23_6.tail(50)

не хватает данных для прогноза на дату 2024-08-10 00:00:00
не хватает данных для прогноза на дату 2024-08-28 00:00:00


month  dist  target  targ_price  region_id  crops_id  \
date       id                                                            
2024-07-23 48874      7   157   70000  445.859873         23         6   
2024-07-25 49059      7   281  130000  462.633452         23         6   
           49085      7   281  130000  462.633452         23         6   
2024-07-26 49168      7   239  130000  543.933054         23         6   
2024-07-27 48989      7   384  170000  442.708333         23         6   
           49221      7   317  118000  372.239748         23         6   
           49267      7   371  170000  458.221024         23         6   
           49270      7   392  175000  446.428571         23         6   
2024-07-28 49212      7   263  105000  399.239544         23         6   
           49242      7   288  110000  381.944444         23         6   
           49302      7   288  112000  388.888889         23         6   
2024-07-29 49213      7   246  100000  406.504065         23         6   
2024-07-30 48890      7   317  115000  362.776025         23         6   
           49304      7   315  112000  355.555556         23         6   
           49318      7   404  175000  433.168317         23         6   
           49335      7   263  105000  399.239544         23         6   
           49343      7   220   85000  386.363636         23         6   
           49363      7   100   75000  750.000000         23         6   
2024-07-31 49336      7   246  100000  406.504065         23         6   
           49364      7   375  165000  440.000000         23         6   
           49374      7   376  165000  438.829787         23         6   
2024-08-01 49488      8   189  112500  595.238095         23         6   
2024-08-02 49390      8   335  160000  477.611940         23         6   
           49547      8   246  103000  418.699187         23         6   
           49559      8   410  144000  351.219512         23         6   
2024-08-04 49656      8   246  115000  467.479675         23         6   
           49668      8   314  115000  366.242038         23         6   
2024-08-05 49675      8   422  170000  402.843602         23         6   
           49682      8   364  132900  365.109890         23         6   
           49722      8   246  107000  434.959350         23         6   
           49727      8   404  183000  452.970297         23         6   
2024-08-06 49704      8   289  140000  484.429066         23         6   
           49735      8   277  115000  415.162455         23         6   
           49820      8   220  100000  454.545455         23         6   
2024-08-07 49819      8   283  150000  530.035336         23         6   
           49829      8   246  110000  447.154472         23         6   
2024-08-08 49758      8   335  177700  530.447761         23         6   
           49883      8   200  130000  650.000000         23         6   
           49927      8   445  165000  370.786517         23         6   
2024-08-09 49974      8   447  165000  369.127517         23         6   
           49983      8   444  150000  337.837838         23         6   
           50007      8   447  165000  369.127517         23         6   
2024-08-10 49946      8   317  120000  378.548896         23         6   
           49985      8   368  132900  361.141304         23         6   
2024-08-11 50093      8   246  110000  447.154472         23         6   
2024-08-13 50145      8   317  135000  425.867508         23         6   
           50180      8   246  123700  502.845528         23         6   
           50202      8   464  175000  377.155172         23         6   
2024-08-16 50375      8   425  170000  400.000000         23         6   
2024-08-28 51215      8   276  138000  500.000000         23         6   

                  day_of_week  hours  dest_id  var3_pred3 errvar3_pred3  
date       id                                                            
2024-07-23 48874     

In [22]:
var3_23_6.to_csv('var3_23_6.csv',sep = ';')